In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Configuración
num_logs = 60
patinadora_ids_all = [f"FENIX_{i:02d}" for i in range(1, 16)]
estrella_fugaz_id = "FENIX_02"
nova_id = "FENIX_09"

# Casilleros (asumimos que cada patinadora tiene uno, ej. CAS_FENIX_01)
casilleros = {pid: f"CAS_{pid}" for pid in patinadora_ids_all}

start_date_logs = datetime(2024, 8, 1) # Mismo rango que datos principales
end_date_logs = datetime(2024, 10, 31)
time_delta_logs_days = (end_date_logs - start_date_logs).days

data_logs = []
log_id_counter = 1

for _ in range(num_logs):
    pat_acceso = random.choice(patinadora_ids_all)
    fecha_acceso_dt = start_date_logs + timedelta(days=random.randint(0, time_delta_logs_days), 
                                                hours=random.randint(7, 21), 
                                                minutes=random.randint(0, 59))
    fecha_hora_acceso_str = fecha_acceso_dt.strftime('%Y-%m-%d %H:%M:%S')
    
    casillero_obj = casilleros[pat_acceso] # Por defecto, accede a su propio casillero
    duracion_acceso = random.randint(1, 5) # Duración normal

    # Sembrando salseo
    if pat_acceso == nova_id and random.random() < 0.4: # Nova accede al casillero de Estrella
        casillero_obj = casilleros[estrella_fugaz_id]
        duracion_acceso = random.randint(5, 15) # Duración más larga y sospechosa
        # Asegurar que este acceso no coincida con un momento donde Estrella esté presente
        # (Esto es una simplificación, en un caso real necesitaríamos horarios de Estrella)
    elif random.random() < 0.1: # Otro acceso aleatorio a casillero ajeno (ruido)
        casillero_obj = random.choice(list(casilleros.values()))
        while casillero_obj == casilleros[pat_acceso]: # Asegurar que no es el suyo
             casillero_obj = random.choice(list(casilleros.values()))
        duracion_acceso = random.randint(1, 3)


    data_logs.append({
        'ID_Log': f"LOG{log_id_counter:03d}",
        'Fecha_Hora_Acceso': fecha_hora_acceso_str,
        'ID_Patinadora_Acceso': pat_acceso,
        'Casillero_Accedido': casillero_obj,
        'Duracion_Acceso_Min': duracion_acceso
    })
    log_id_counter += 1

df_logs = pd.DataFrame(data_logs)
df_logs.to_csv('vestuario_log_accesos.csv', sep=';', index=False, encoding='utf-8-sig')
print("Archivo 'vestuario_log_accesos.csv' (con salseo) creado exitosamente.")
print(f"Total de logs: {len(df_logs)}")
print("\nLogs sospechosos de Nova accediendo al casillero de EstrellaFugaz:")
print(df_logs[(df_logs['ID_Patinadora_Acceso'] == nova_id) & (df_logs['Casillero_Accedido'] == casilleros[estrella_fugaz_id])].head())
print("\nOtros ejemplos de logs:")
print(df_logs.sample(5))

Archivo 'vestuario_log_accesos.csv' (con salseo) creado exitosamente.
Total de logs: 60

Logs sospechosos de Nova accediendo al casillero de EstrellaFugaz:
Empty DataFrame
Columns: [ID_Log, Fecha_Hora_Acceso, ID_Patinadora_Acceso, Casillero_Accedido, Duracion_Acceso_Min]
Index: []

Otros ejemplos de logs:
    ID_Log    Fecha_Hora_Acceso ID_Patinadora_Acceso Casillero_Accedido  \
33  LOG034  2024-09-15 07:08:00             FENIX_08       CAS_FENIX_08   
29  LOG030  2024-09-10 17:16:00             FENIX_04       CAS_FENIX_04   
7   LOG008  2024-08-20 11:27:00             FENIX_04       CAS_FENIX_04   
55  LOG056  2024-10-16 14:31:00             FENIX_13       CAS_FENIX_13   
11  LOG012  2024-10-20 08:28:00             FENIX_14       CAS_FENIX_14   

    Duracion_Acceso_Min  
33                    4  
29                    1  
7                     1  
55                    3  
11                    2  


In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Configuración
num_rows_material = 40
patinadora_ids = [f"FENIX_{i:02d}" for i in range(1, 16)] # FENIX_01 a FENIX_15
start_date = datetime(2024, 8, 1) # Coincidir con el rango de fechas del dataset principal
end_date = datetime(2024, 10, 31)
time_delta_days = (end_date - start_date).days

tipos_articulo = [
    "Patines Competicion Alpha", "Ruedas Dureza 92A", "Protecciones Rodilla Pro",
    "Botas Modelo Elite", "Cuchillas Figura Master", "Patines Entrenamiento Beta",
    "Ruedas Agarre Extremo 88A", "Casco Aero Dinamic", "Guantes Reforzados Kevlar",
    "Vestuario Gala Brillante", "Cordones Encerados Sport"
]
desgaste_opciones = ["Leve", "Medio", "Ninguno", "Alto", np.nan, "", "N/A", "Medio-Alto"]
horas_uso_formatos = [lambda x: f"{x:.1f}", lambda x: f"{x:.0f}h", lambda x: f"{int(x*60)} min", lambda x: f"{x:.1f}".replace('.',','), lambda x: np.nan]

data_material = []
for i in range(1, num_rows_material + 1):
    pat_id = random.choice(patinadora_ids)
    sesion_date_dt = start_date + timedelta(days=random.randint(0, time_delta_days))
    fecha_sesion_str = sesion_date_dt.strftime('%Y-%m-%d') # Formato consistente para unir
    
    articulo = random.choice(tipos_articulo)
    horas_base = round(random.uniform(0.5, 3.0), 1)
    formato_horas = random.choice(horas_uso_formatos)
    horas_uso_str = formato_horas(horas_base) if not (isinstance(formato_horas(horas_base), float) and np.isnan(formato_horas(horas_base))) else np.nan


    desgaste = random.choice(desgaste_opciones)
    
    data_material.append({
        'ID_Sesion_Material': f"MAT{i:03d}",
        'ID_Patinadora': pat_id,
        'Fecha_Sesion': fecha_sesion_str,
        'Tipo_Articulo_Usado': articulo,
        'Horas_Uso_Articulo_Sesion': horas_uso_str,
        'Desgaste_Reportado': desgaste
    })

df_material_gen = pd.DataFrame(data_material)
df_material_gen.to_csv('fenix_uso_material_sesion.csv', sep=';', index=False, encoding='utf-8-sig')
print("Archivo 'fenix_uso_material_sesion.csv' creado exitosamente.")
print(df_material_gen.head())

Archivo 'fenix_uso_material_sesion.csv' creado exitosamente.
  ID_Sesion_Material ID_Patinadora Fecha_Sesion        Tipo_Articulo_Usado  \
0             MAT001      FENIX_11   2024-10-29   Protecciones Rodilla Pro   
1             MAT002      FENIX_07   2024-09-18   Cordones Encerados Sport   
2             MAT003      FENIX_05   2024-08-07   Protecciones Rodilla Pro   
3             MAT004      FENIX_03   2024-10-23  Guantes Reforzados Kevlar   
4             MAT005      FENIX_09   2024-10-05    Cuchillas Figura Master   

  Horas_Uso_Articulo_Sesion Desgaste_Reportado  
0                        1h            Ninguno  
1                       1,0         Medio-Alto  
2                       NaN                     
3                       NaN               Alto  
4                       NaN               Leve  


In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Configuración
num_rows_animo = 40
patinadora_ids = [f"FENIX_{i:02d}" for i in range(1, 16)]
start_date = datetime(2024, 8, 1)
end_date = datetime(2024, 10, 31)
time_delta_days = (end_date - start_date).days

molestias_opciones = ["Ninguna", "Tobillo Izq.", "Rodilla leve", "", np.nan, "Cansancio General", "Espalda Baja", "Hombro Derecho", "N/A", "Sin quejas"]
energia_foco_opciones = [1, 2, 3, 4, 5, np.nan, "3 (OK)", "N/A"]


data_animo = []
for i in range(1, num_rows_animo + 1):
    pat_id = random.choice(patinadora_ids)
    sesion_date_dt = start_date + timedelta(days=random.randint(0, time_delta_days))
    fecha_sesion_str = sesion_date_dt.strftime('%Y-%m-%d')

    energia = random.choice(energia_foco_opciones)
    foco = random.choice(energia_foco_opciones)
    molestia = random.choice(molestias_opciones)

    data_animo.append({
        'ID_Reporte_Animo': f"ANM{i:03d}",
        'ID_Patinadora': pat_id,
        'Fecha_Sesion': fecha_sesion_str,
        'Nivel_Energia_PreSesion': energia,
        'Foco_Mental_PreSesion': foco,
        'Molestia_Fisica_Previa': molestia
    })

df_animo_gen = pd.DataFrame(data_animo)
df_animo_gen.to_csv('fenix_estado_animo_patinadora.csv', sep=';', index=False, encoding='utf-8-sig')
print("Archivo 'fenix_estado_animo_patinadora.csv' creado exitosamente.")
print(df_animo_gen.head())

Archivo 'fenix_estado_animo_patinadora.csv' creado exitosamente.
  ID_Reporte_Animo ID_Patinadora Fecha_Sesion Nivel_Energia_PreSesion  \
0           ANM001      FENIX_02   2024-10-25                     N/A   
1           ANM002      FENIX_10   2024-09-14                  3 (OK)   
2           ANM003      FENIX_10   2024-08-30                       5   
3           ANM004      FENIX_04   2024-08-09                  3 (OK)   
4           ANM005      FENIX_03   2024-09-17                     N/A   

  Foco_Mental_PreSesion Molestia_Fisica_Previa  
0                     5                         
1                     5           Espalda Baja  
2                     2             Sin quejas  
3                     5                Ninguna  
4                   N/A           Rodilla leve  
